In [0]:
import json
import logging
from azureml.core import Workspace, Dataset
import azureml.core
import pandas as pd
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
import matplotlib.pyplot as plt
from matplotlib import rcParams
from cycler import cycler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns

In [0]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [0]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = "automl-ojforecasting"

experiment = Experiment(ws, experiment_name)

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["SKU"] = ws.sku
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

In [0]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "Forcast-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D12_V2", max_nodes=6
    )
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [0]:
df = pd.read_csv("./Dataset.csv", delimiter=";")
df.head()

In [0]:
df.info()

In [0]:
df=df.drop(['FY','POST_PERIOD','Num of BR','Num Organisation'], axis=1)
df.head()

In [0]:
plt.figure(figsize=(16,5))
plt.plot(df['Timestamp'],df['Revenue'])
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.title("revenue per periods")
plt.show()

In [0]:
df['Revenue_diff']=df['Revenue'].diff()
df=df.dropna()
df.head(10)

In [0]:
plt.figure(figsize=(16,5))
plt.plot(df['Timestamp'],df['Revenue_diff'])
plt.xlabel("Date")
plt.ylabel("Revenue_diff")
plt.title("revenue diff per periods")
plt.show()